In [ ]:
import numpy as np
import tensorflow as tf
import random
import tensorflow_datasets as tfds
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import os
import sys
from datetime import datetime

In [ ]:
# Work-around for loading a module from a parent folder in Jupyter/Notebooks
parent_dir = os.path.abspath(os.path.join('..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
from modules.autoencoders import VanillaAutoencoder, ConvolutedAutoencoder, VariationalAutoencoder

In [ ]:
# Caricamento dei dati
#(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Caricamento dei dati
train_ds, test_ds = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, batch_size=-1, as_supervised=True)
x_train, y_train = tfds.as_numpy(train_ds)
x_test, y_test = tfds.as_numpy(test_ds)
#x_train = x_train[:,:,:,0]
#x_test = x_test[:,:,:,0]

In [ ]:
# Normalizzazione dei pixel a [0, 1]
norm_x_train = x_train.astype('float32') / 255.
norm_x_test = x_test.astype('float32') / 255.

# Dataset Analisys

In [ ]:
# Troviamo le classi presenti (da 0 a 9)
classes = np.unique(y_train)
n_classes = len(classes)
samples_per_class = 10  # Numero di esempi per classe da visualizzare

# Creiamo una griglia di subplot: righe = n_classes, colonne = samples_per_class
fig, axes = plt.subplots(n_classes, samples_per_class, figsize=(samples_per_class * 1, n_classes * 1))

for i, cls in enumerate(classes):
    # Trova gli indici degli esempi per la classe corrente
    idxs = np.where(y_train == cls)[0]    
    # Mostra l'etichetta sulla prima colonna di ogni riga
    axes[i, 0].set_title(f"Class {cls}")
    j=0    
    # Seleziona randomicamente samples_per_class indici tra quelli della classe corrente
    for idx in random.sample(list(idxs), samples_per_class):
        # Seleziona l'immagine
        img = x_train[idx]
        # Visualizza l'immagine
        axes[i, j].imshow(img, cmap='gray')
        axes[i, j].axis('off')
        j=j+1

plt.tight_layout()
plt.show()

# Utils

In [ ]:
def plot_img(immagine, titolo="Sampled Image", figsize=(4,4)):
    """
    Plotta una immagine, convertendola se necessario da vettoriale (784,) a matrice (28,28).

    Parametri:
    - immagine: numpy array rappresentante l'immagine (forma (784,) o (28,28))
    - titolo: titolo da visualizzare sul plot
    """
    # Se l'immagine è un vettore piatto, la rimodelliamo in 28x28
    if immagine.ndim == 1 or (immagine.ndim == 2 and immagine.shape[0] * immagine.shape[1] == 784):
        immagine = immagine.reshape(28, 28)

    plt.figure(figsize=figsize)
    plt.imshow(immagine, cmap='gray')
    plt.title(titolo)
    plt.axis('off')
    plt.show()

In [ ]:
def plot_encoding_tsne(encoder, data, labels, sample_size=None, perplexity=30, random_state=42, title="t-SNE projection of the latent space"):
    """
    Calcola l'encoding dei dati con il modello encoder e proietta le rappresentazioni latenti in 2D
    utilizzando t-SNE. I punti sono colorati in modo discreto in base alle etichette (classi).

    Parametri:
    - encoder: modello Keras per ottenere l'encoding.
    - dati: array dei dati in input (es. immagini).
    - labels: array delle etichette corrispondenti.
    - sample_size: (opzionale) numero di campioni da utilizzare per t-SNE (se None usa tutti i dati).
    - perplexity: parametro perplexity per t-SNE.
    - random_state: seme per la riproducibilità.
    """
    # Se sample_size è definito e minore del numero totale di dati, campiona casualmente
    if sample_size is not None and sample_size < len(data):
        idx = np.random.choice(len(data), size=sample_size, replace=False)
        data = data[idx]
        labels = labels[idx]

    # Calcola l'encoding dei dati
    _predicted_ = encoder.predict(data)
    if type(_predicted_) == list:
        latent = _predicted_[0]
    else:
        latent = _predicted_

    # Riduci a 2 dimensioni con t-SNE
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=random_state)
    latent_2d = tsne.fit_transform(latent)

    # Ottieni le etichette uniche
    unique_labels = np.unique(labels)
    # Se ci sono al massimo 10 classi, usiamo 'tab10', altrimenti 'tab20'
    if len(unique_labels) <= 10:
        cmap = plt.get_cmap('tab10', len(unique_labels))
    else:
        cmap = plt.get_cmap('tab20', len(unique_labels))

    plt.figure(figsize=(8, 6))
    # Plot separato per ogni classe con colori distinti
    for i, ul in enumerate(unique_labels):
        idx = labels == ul
        plt.scatter(latent_2d[idx, 0], latent_2d[idx, 1],
                    color=cmap(i), label=str(ul), s=5)

    plt.title(title)
    plt.xlabel("t-SNE 1")
    plt.ylabel("t-SNE 2")
    plt.legend(title='Classe')
    plt.show()


In [ ]:
#model_name = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = "20250320_175412"
model_path = os.path.join("..", "models")

# Vanilla Autoencoder

In [ ]:
flatten_x_train = norm_x_train.reshape((len(norm_x_train), -1))
flatten_x_test = norm_x_test.reshape((len(norm_x_test), -1))

In [ ]:
input_dim = flatten_x_train.shape[1]
neurons=(128, 64, 32)
activations=('relu', 'relu', 'relu')
latent_space_dim=2

vannilla_model_type = VanillaAutoencoder.__name__

vanilla_model_params_path = os.path.join(model_path, f"{vannilla_model_type}_{model_name}.parameters.pkl")
vanilla_model_weights_path = os.path.join(model_path, f"{vannilla_model_type}_{model_name}.weights.h5")
if not os.path.exists(vanilla_model_params_path) or not os.path.exists(vanilla_model_weights_path):
    vanilla_model = VanillaAutoencoder(input_dim=input_dim, neurons=neurons, activations=activations, latent_space_dim=latent_space_dim, model_name=model_name)
    optimizer = tf.keras.optimizers.get("Adam")
    optimizer.learning_rate.assign(.0001)
    vanilla_model.compile(optimizer=optimizer, loss='mse')
else:
    vanilla_model = VanillaAutoencoder.load(model_path, model_name)

vanilla_model.print_summaries()

In [ ]:
if not os.path.exists(vanilla_model_params_path) or not os.path.exists(vanilla_model_weights_path):
    history = vanilla_model.train(
                flatten_x_train, 
                flatten_x_test,
                epochs=10,
                batch_size=512,
                shuffle=True)
else:
    history = vanilla_model.get_training_history()

In [ ]:
if not os.path.exists(vanilla_model_params_path) or not os.path.exists(vanilla_model_weights_path):
    vanilla_model.save(model_path)

In [ ]:
vanilla_encoder = vanilla_model.get_encoder()
vanilla_decoder = vanilla_model.get_decoder()
vanilla_autoencoder = vanilla_model

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss (Vanilla)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# -------------------------------
# Calcolo delle statistiche dello spazio latente
# (utilizzato per il sampling)
# -------------------------------
latent_reps = vanilla_encoder.predict(flatten_x_train)

In [ ]:
latent_mean = np.mean(latent_reps, axis=0)
latent_std = np.std(latent_reps, axis=0)

In [ ]:
encoded_img = np.random.normal(loc=latent_mean, scale=latent_std, size=(1,latent_space_dim))
# Decodifica per ottenere la ricostruzione
decoded_img = vanilla_decoder.predict(encoded_img)
# Genera una nuova immagine
plot_img(decoded_img, figsize=(1,1))

In [ ]:
plot_encoding_tsne(vanilla_encoder, flatten_x_test, y_test, sample_size=1000, title=f"t-SNE projection of the latent space (Vanilla Encoder)")

# Convoluted Autoencoder

In [ ]:
conv_input_shape = (norm_x_train.shape[1:][0], norm_x_train.shape[1:][1], 1)
conv_filters=(32, 64, 64, 64)
conv_activations=('relu', 'relu', 'relu', 'relu')
conv_kernels=(3, 3, 3, 3)
conv_strides=(1, 2, 2, 1)
conv_latent_space_dim=2

conv_model_type = ConvolutedAutoencoder.__name__

conv_model_params_path = os.path.join(model_path, f"{conv_model_type}_{model_name}.parameters.pkl")
conv_model_weights_path = os.path.join(model_path, f"{conv_model_type}_{model_name}.weights.h5")
if not os.path.exists(conv_model_params_path) or not os.path.exists(conv_model_weights_path):
    conv_model = ConvolutedAutoencoder(
        input_shape=conv_input_shape, 
        filters=conv_filters, 
        kernels=conv_kernels, 
        strides=conv_strides, 
        activations=conv_activations,
        latent_space_dim=conv_latent_space_dim,
        model_name=model_name
    )
    optimizer = tf.keras.optimizers.get("Adam")
    optimizer.learning_rate.assign(.0001)
    conv_model.compile(optimizer=optimizer, loss='mse')
else:
    conv_model = ConvolutedAutoencoder.load(model_path, model_name)

conv_model.print_summaries()

In [ ]:
#tf.config.run_functions_eagerly(False)
if not os.path.exists(conv_model_params_path) or not os.path.exists(conv_model_weights_path):
    conv_history = conv_model.train(
                norm_x_train, 
                norm_x_test,
                epochs=10,
                batch_size=512,
                shuffle=True
            )
else:
    conv_history = conv_model.get_training_history()

In [ ]:
if not os.path.exists(conv_model_params_path) or not os.path.exists(conv_model_weights_path):
    conv_model.save(model_path)

In [ ]:
conv_encoder = conv_model.get_encoder()
conv_decoder = conv_model.get_decoder()
conv_autoencoder = conv_model

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(conv_history.history['loss'])
plt.plot(conv_history.history['val_loss'])
plt.title('Model loss (Convoluted)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# -------------------------------
# Calcolo delle statistiche dello spazio latente
# (utilizzato per il sampling)
# -------------------------------
conv_latent_reps = conv_encoder.predict(norm_x_train)

In [ ]:
conv_latent_mean = np.mean(conv_latent_reps, axis=0)
conv_latent_std = np.std(conv_latent_reps, axis=0)

In [ ]:
conv_encoded_img = np.random.normal(loc=conv_latent_mean, scale=conv_latent_std, size=(1,latent_space_dim))
# Decodifica per ottenere la ricostruzione
conv_decoded_img = conv_decoder.predict(conv_encoded_img)
# Genera una nuova immagine
plot_img(conv_decoded_img[0], figsize=(1,1))

In [ ]:
plot_encoding_tsne(conv_encoder, norm_x_test, y_test, sample_size=1000, title=f"t-SNE projection of the latent space (Convoluted)")

# Variational Autoencoder

### Loss Function for Variational Autoencoders

$Loss = \alpha \cdot RMSE + KL$

* $RMSE$  
Root mean squared error. It's the Reconstruction error
* $\alpha$ 
Reconstruction loss weight
* $KL$  
Kullback-Leibler divergence. It measures the _difference_ between a multivariate normal distribution and a multivariate standard normal distribution.  
A standard normal distribution is a normal distribution with mean ($\mu$) equal to 0 and standard deviation ($\sigma$) equal to 1

Kullback-Leibler divergence (Closed-form)  
$D_{KL} (N(\mu,\sigma) || N(0,1))= \frac{1}{2} \sum \left(1+\log(\sigma^2)-\mu^2-\sigma^2\right)$

In [ ]:
vae_input_shape = (norm_x_train.shape[1:][0], norm_x_train.shape[1:][1], 1)
vae_filters=(32, 64, 64, 64)
vae_activations=('relu', 'relu', 'relu', 'relu')
vae_kernels=(3, 3, 3, 3)
vae_strides=(1, 2, 2, 1)
vae_latent_space_dim=2

vae_model_type = VariationalAutoencoder.__name__

vae_model_params_path = os.path.join(model_path, f"{vae_model_type}_{model_name}.parameters.pkl")
vae_model_weights_path = os.path.join(model_path, f"{vae_model_type}_{model_name}.weights.h5")
if not os.path.exists(vae_model_params_path) or not os.path.exists(vae_model_weights_path):
    vae_model = VariationalAutoencoder(
        input_shape=vae_input_shape, 
        filters=vae_filters, 
        kernels=vae_kernels, 
        strides=vae_strides, 
        activations=vae_activations,
        latent_space_dim=vae_latent_space_dim,
        model_name=model_name,
        reconstruction_loss_weight=1
    )
    optimizer = tf.keras.optimizers.get("Adam")
    optimizer.learning_rate.assign(.001)
    vae_model.compile(optimizer=optimizer)
else:
    vae_model = VariationalAutoencoder.load(model_path, model_name)

#vae_model.print_summaries()

In [ ]:
#tf.config.run_functions_eagerly(False)
if not os.path.exists(vae_model_params_path) or not os.path.exists(vae_model_weights_path):
    vae_history = vae_model.train(
                norm_x_train, 
                norm_x_test,
                epochs=20,
                batch_size=512,
                shuffle=True
            )
else:
    vae_history = vae_model.get_training_history()

In [ ]:
if not os.path.exists(vae_model_params_path) or not os.path.exists(vae_model_weights_path):
    vae_model.save(model_path)

In [ ]:
vae_encoder = vae_model.get_encoder()
vae_decoder = vae_model.get_decoder()
vae_autoencoder = vae_model

In [ ]:
# Plot training & validation loss values
plt.figure(figsize=(10, 5))
plt.plot(vae_history.history['loss'])
plt.plot(vae_history.history['val_loss'])
plt.title('Model loss (VAE)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# -------------------------------
# Calcolo delle statistiche dello spazio latente
# (utilizzato per il sampling)
# -------------------------------
vae_latent_reps, _, _ = vae_encoder.predict(norm_x_train)

In [ ]:
vae_latent_mean = np.mean(vae_latent_reps, axis=0)
vae_latent_std = np.std(vae_latent_reps, axis=0)

In [ ]:
vae_encoded_img = np.random.normal(loc=vae_latent_mean, scale=vae_latent_std, size=(1,latent_space_dim))
# Decodifica per ottenere la ricostruzione
vae_decoded_img = vae_decoder.predict(vae_encoded_img)
# Genera una nuova immagine
plot_img(vae_decoded_img[0], figsize=(1,1))

In [ ]:
plot_encoding_tsne(vae_encoder, norm_x_test, y_test, sample_size=1000, title=f"t-SNE projection of the latent space (Variational Encoder)")

# Latent Spaces Analysis

In [ ]:
plot_encoding_tsne(vanilla_encoder, flatten_x_train, y_train, sample_size=1000, title=f"t-SNE projection of the latent space (Vanilla Encoder)")

In [ ]:
plot_encoding_tsne(conv_encoder, norm_x_train, y_train, sample_size=1000, title=f"t-SNE projection of the latent space (Convoluted Encoder)")

In [ ]:
plot_encoding_tsne(vae_encoder, norm_x_train, y_train, sample_size=1000, title=f"t-SNE projection of the latent space (Variational Encoder)")